In [2]:
import pandas as pd
import os
import json

# Define the path to the output folder
output_folder = '/workspaces/cwl-data/output/structured-2018-04-08-proleague1'

# Initialize an empty list to store the data
data = []

# Loop through all files in the output folder
for filename in os.listdir(output_folder):
    if filename.endswith('.json'):
        file_path = os.path.join(output_folder, filename)
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            data.append(json_data)

# Create a DataFrame from the list of data
df = pd.DataFrame(data)

# Display the DataFrame
print(df.head())

  title platform                                    id   series_id  \
0   ww2      ps4  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10   
1   ww2      ps4  ab7e12d2-8217-5561-b6c8-927431692315   pro1-a2-7   
2   ww2      ps4  f8898cb4-cd99-5083-9d23-4199baa63001   pro1-b4-4   
3   ww2      ps4               missing-pro1-a4-7-map-3   pro1-a4-7   
4   ww2      ps4  7a620e83-ce6b-5286-876f-74c81d7952f9   pro1-b1-9   

   start_time_s  end_time_s  duration_ms              mode  \
0    1519269582  1519270242       660000  Capture The Flag   
1    1517439951  1517440618       667000         Hardpoint   
2    1521597507  1521598290       783000  Capture The Flag   
3    1519860288  1519860888       600000  Capture The Flag   
4    1518052699  1518053000       301000  Search & Destroy   

                    map  rounds  \
0       Ardennes Forest       2   
1  Sainte Marie du Mont       1   
2            Flak Tower       4   
3       Ardennes Forest       1   
4       Ardennes Forest       7

In [3]:
expanded_rows = []

for _, row in df.iterrows():
    title = row['title']
    map_name = row['map']
    mode = row['mode']
    game_id = row['id']
    series_id = row['series_id']
    start_time_s = row['start_time_s']
    
    for event in row['events']:
        if event['type'] == 'death':
            event_data = event['data']
            expanded_row = {
                'title': title,
                'map': map_name,
                'mode': mode,
                'id': game_id,
                'series_id': series_id,
                'start_time_s': start_time_s,
                'event_type': event['type'],
                'time_ms': event['time_ms'],
                'victim_id': event_data['id'],
                'attacker_id': event_data['attacker']['id'],
                'pos_x': event_data['pos']['x'],
                'pos_y': event_data['pos']['y'],
                'attacker_pos_x': event_data['attacker']['pos']['x'],
                'attacker_pos_y': event_data['attacker']['pos']['y'],
                'means_of_death': event_data['attacker']['means_of_death']
            }
            expanded_rows.append(expanded_row)

expanded_df = pd.DataFrame(expanded_rows)
print(expanded_df.head())

  title              map              mode  \
0   ww2  Ardennes Forest  Capture The Flag   
1   ww2  Ardennes Forest  Capture The Flag   
2   ww2  Ardennes Forest  Capture The Flag   
3   ww2  Ardennes Forest  Capture The Flag   
4   ww2  Ardennes Forest  Capture The Flag   

                                     id   series_id  start_time_s event_type  \
0  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582      death   
1  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582      death   
2  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582      death   
3  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582      death   
4  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582      death   

   time_ms victim_id attacker_id  pos_x  pos_y  attacker_pos_x  \
0        0      JOEE        JOEE  240.0  364.0           240.0   
1        0   GUNLESS     GUNLESS  850.0  583.0           850.0   
2     9400      JOEE     GUNLESS  609.0  425

In [4]:
expanded_players = []

for _, row in df.iterrows():
    title = row['title']
    map_name = row['map']
    mode = row['mode']
    game_id = row['id']
    series_id = row['series_id']
    start_time_s = row['start_time_s']
    
    for player in row['players']:
        expanded_player = {
            'title': title,
            'map': map_name,
            'mode': mode,
            'id': game_id,
            'series_id': series_id,
            'start_time_s': start_time_s,
            'player_id': player['name'],
            'fave_weapon': player['fave_weapon']
        }
        expanded_players.append(expanded_player)

expanded_players_df = pd.DataFrame(expanded_players)
print(expanded_players_df.head())

  title              map              mode  \
0   ww2  Ardennes Forest  Capture The Flag   
1   ww2  Ardennes Forest  Capture The Flag   
2   ww2  Ardennes Forest  Capture The Flag   
3   ww2  Ardennes Forest  Capture The Flag   
4   ww2  Ardennes Forest  Capture The Flag   

                                     id   series_id  start_time_s player_id  \
0  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582   GUNLESS   
1  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582     LOONY   
2  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582   METHODZ   
3  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582    TJHALY   
4  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582      JOEE   

  fave_weapon  
0      STG-44  
1     PPSh-41  
2      STG-44  
3     PPSh-41  
4     PPSh-41  


In [5]:
# Select specific columns from the expanded dataframe
selected_columns = ['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'attacker_id', 'time_ms']

# Create a new dataset with the selected columns
attacker_list_df = expanded_df[selected_columns]

# Display the first few rows of the new dataset
print(attacker_list_df.head())

  title              map              mode  \
0   ww2  Ardennes Forest  Capture The Flag   
1   ww2  Ardennes Forest  Capture The Flag   
2   ww2  Ardennes Forest  Capture The Flag   
3   ww2  Ardennes Forest  Capture The Flag   
4   ww2  Ardennes Forest  Capture The Flag   

                                     id   series_id  start_time_s attacker_id  \
0  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582        JOEE   
1  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582     GUNLESS   
2  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582     GUNLESS   
3  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582      TJHALY   
4  e2c58665-e66c-54a6-af6e-f7658c88367c  pro1-a3-10    1519269582        ZER0   

   time_ms  
0        0  
1        0  
2     9400  
3    11800  
4    12250  


In [19]:
## create a copy of expanded_df 
was_victim_killer_in_prev_5000ms = expanded_df.copy()

## join the copy with attacker_list_df on title, map, mode, id, series_id, start_time_s, victim_id = attacker_id
was_victim_killer_in_prev_5000ms = was_victim_killer_in_prev_5000ms.merge(
    attacker_list_df,
    left_on=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'victim_id'],
    right_on=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'attacker_id'],
    how='inner',
    suffixes=('', '_kill')
)

## filter where time_ms_kill between time_ms and time_ms - 5000
was_victim_killer_in_prev_5000ms = was_victim_killer_in_prev_5000ms[
    (was_victim_killer_in_prev_5000ms['time_ms_kill'] < was_victim_killer_in_prev_5000ms['time_ms']) &
    (was_victim_killer_in_prev_5000ms['time_ms_kill'] > was_victim_killer_in_prev_5000ms['time_ms'] - 5000)
]

## now get distinct rows by title, map, mode, id, series_id, start_time_s, victim_id, and time_ms
was_victim_killer_in_prev_5000ms = was_victim_killer_in_prev_5000ms.drop_duplicates(
    subset=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'victim_id', 'time_ms'],
    ignore_index=True
)[['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'victim_id', 'time_ms']]

## add a column called victim_kill_in_prev_5000ms and set it to true
was_victim_killer_in_prev_5000ms['victim_kill_in_prev_5000ms'] = True

# ## head
# was_victim_killer_in_prev_5000ms.head()



# Create a copy of expanded_df
was_attacker_killer_in_prev_5000ms = expanded_df.copy()

# Join the copy with attacker_list_df on title, map, mode, id, series_id, start_time_s, attacker_id = attacker_id
was_attacker_killer_in_prev_5000ms = was_attacker_killer_in_prev_5000ms.merge(
    attacker_list_df,
    left_on=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'attacker_id'],
    right_on=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'attacker_id'],
    how='inner',
    suffixes=('', '_kill')
)

# Filter where time_ms_kill between time_ms and time_ms - 5000
was_attacker_killer_in_prev_5000ms = was_attacker_killer_in_prev_5000ms[
    (was_attacker_killer_in_prev_5000ms['time_ms_kill'] < was_attacker_killer_in_prev_5000ms['time_ms']) &
    (was_attacker_killer_in_prev_5000ms['time_ms_kill'] > was_attacker_killer_in_prev_5000ms['time_ms'] - 5000)
]

# Now get distinct rows by title, map, mode, id, series_id, start_time_s, attacker_id, and time_ms
was_attacker_killer_in_prev_5000ms = was_attacker_killer_in_prev_5000ms.drop_duplicates(
    subset=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'attacker_id', 'time_ms'],
    ignore_index=True
)[['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'attacker_id', 'time_ms']]

# Add a column called attacker_kill_in_prev_5000ms and set it to true
was_attacker_killer_in_prev_5000ms['attacker_kill_in_prev_5000ms'] = True

# # Display the first few rows of the new dataframe
# was_attacker_killer_in_prev_5000ms.head()

## first merge was_victim_killer_in_prev_5000ms with expanded_df on title, map, mode, id, series_id, start_time_s, victim_id, time_ms
expanded_df_with_prev_eng = expanded_df.merge(
    was_victim_killer_in_prev_5000ms,
    on=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'victim_id', 'time_ms'],
    how='left'
)

## then merge was_attacker_killer_in_prev_5000ms with expanded_df_with_prev_eng on title, map, mode, id, series_id, start_time_s, attacker_id, time_ms
expanded_df_with_prev_eng = expanded_df_with_prev_eng.merge(
    was_attacker_killer_in_prev_5000ms,
    on=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'attacker_id', 'time_ms'],
    how='left'
)

## fill NaN values in victim_kill_in_prev_5000ms and attacker_kill_in_prev_5000ms with False
expanded_df_with_prev_eng['victim_kill_in_prev_5000ms'] = expanded_df_with_prev_eng['victim_kill_in_prev_5000ms'].fillna(False)
expanded_df_with_prev_eng['attacker_kill_in_prev_5000ms'] = expanded_df_with_prev_eng['attacker_kill_in_prev_5000ms'].fillna(False)

## now join in expanded_players_df on victim_id = player_id to get fave_weapon
expanded_df_with_prev_eng = expanded_df_with_prev_eng.merge(
    expanded_players_df,
    left_on=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'victim_id'],
    right_on=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'player_id'],
    how='left'
)

## drop cols
expanded_df_with_prev_eng = expanded_df_with_prev_eng.drop(
    columns=['player_id']
)

# Reorder expanded_df_with_prev_eng to sort by title, map, mode, id, series_id, start_time_s, and time_ms
expanded_df_with_prev_eng = expanded_df_with_prev_eng.sort_values(
    by=['title', 'map', 'mode', 'id', 'series_id', 'start_time_s', 'time_ms']
)

# Display the first few rows of the sorted dataframe
expanded_df_with_prev_eng.head()


/tmp/ipykernel_49807/736131452.py:78: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  expanded_df_with_prev_eng['victim_kill_in_prev_5000ms'] = expanded_df_with_prev_eng['victim_kill_in_prev_5000ms'].fillna(False)
/tmp/ipykernel_49807/736131452.py:79: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  expanded_df_with_prev_eng['attacker_kill_in_prev_5000ms'] = expanded_df_with_prev_eng['attacker_kill_in_prev_5000ms'].fillna(False)


,title,map,mode,id,series_id,start_time_s,event_type,time_ms,victim_id,attacker_id,pos_x,pos_y,attacker_pos_x,attacker_pos_y,means_of_death,victim_kill_in_prev_5000ms,attacker_kill_in_prev_5000ms,fave_weapon
24568,ww2,Ardennes Forest,Capture The Flag,00099d3b-f31b-5a26-b899-de3a90cab761,pro1-a2-5,1517364495,death,0,DENZ,DENZ,848.0,638.0,848.0,638.0,suicide,False,False,FG 42
24569,ww2,Ardennes Forest,Capture The Flag,00099d3b-f31b-5a26-b899-de3a90cab761,pro1-a2-5,1517364495,death,8800,FIGHTA,KENNY,345.0,724.0,606.0,720.0,rifle_bullet,False,False,PPSh-41
24570,ww2,Ardennes Forest,Capture The Flag,00099d3b-f31b-5a26-b899-de3a90cab761,pro1-a2-5,1517364495,death,22200,THEORY,BUZZO,449.0,468.0,474.0,260.0,rifle_bullet,False,False,PPSh-41
24571,ww2,Ardennes Forest,Capture The Flag,00099d3b-f31b-5a26-b899-de3a90cab761,pro1-a2-5,1517364495,death,30450,KENNY,DENZ,505.0,550.0,512.0,742.0,rifle_bullet,False,False,PPSh-41
24572,ww2,Ardennes Forest,Capture The Flag,00099d3b-f31b-5a26-b899-de3a90cab761,pro1-a2-5,1517364495,death,31550,ACCURACY,BUZZO,598.0,418.0,489.0,284.0,rifle_bullet,False,False,FG 42


In [23]:
# Filter the expanded dataframe to include only specific means of death and mode 'Hardpoint'
filtered_df = expanded_df_with_prev_eng[
    (expanded_df_with_prev_eng['means_of_death'].isin(['pistol_bullet', 'rifle_bullet', 'head_shot', 'melee'])) &
    (expanded_df_with_prev_eng['mode'] == 'Hardpoint')
]

# Display the first few rows of the filtered dataframe
print(filtered_df.head())

     title              map       mode                                    id  \
5484   ww2  Ardennes Forest  Hardpoint  31e55d08-8620-5f57-a5f6-bb11cb54ae18   
5485   ww2  Ardennes Forest  Hardpoint  31e55d08-8620-5f57-a5f6-bb11cb54ae18   
5486   ww2  Ardennes Forest  Hardpoint  31e55d08-8620-5f57-a5f6-bb11cb54ae18   
5487   ww2  Ardennes Forest  Hardpoint  31e55d08-8620-5f57-a5f6-bb11cb54ae18   
5488   ww2  Ardennes Forest  Hardpoint  31e55d08-8620-5f57-a5f6-bb11cb54ae18   

      series_id  start_time_s event_type  time_ms victim_id attacker_id  \
5484  pro1-b4-2    1521586498      death     7350     REEDY       ACHES   
5485  pro1-b4-2    1521586498      death     8600    VORTEX    NAMELESS   
5486  pro1-b4-2    1521586498      death    10400     ACHES      HAWQEH   
5487  pro1-b4-2    1521586498      death    10700     DQVEE      ENABLE   
5488  pro1-b4-2    1521586498      death    12450    HAWQEH      APATHY   

      pos_x  pos_y  attacker_pos_x  attacker_pos_y means_of_death  \

In [24]:
means_of_death_counts = filtered_df['fave_weapon'].value_counts()
print(means_of_death_counts)

fave_weapon
PPSh-41    29332
STG-44      5978
FG 42       5051
BAR          560
Name: count, dtype: int64
